# Part 1-Pyspark DataFrames


In [ ]:
trips_df = spark.read.csv('gs://mhl-exam/data.loans2 (1).csv',header=True, inferSchema=True )mhl-exam/data/loans2 (1).csv

In [2]:
trips_df.head()

Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2020, 1, 1, 0, 28, 15), tpep_dropoff_datetime=datetime.datetime(2020, 1, 1, 0, 33, 3), passenger_count=1, trip_distance=1.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=238, DOLocationID=239, payment_type=1, fare_amount=6.0, extra=3.0, mta_tax=0.5, tip_amount=1.47, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=11.27, congestion_surcharge=2.5)

In [3]:
trips_df.show(20)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|          1.2|         1|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.47|         0.0|                  0.3

In [ ]:
# Question 1

In [4]:
trips_df.agg({'trip_distance': 'mean'}).show( )

+------------------+
|avg(trip_distance)|
+------------------+
| 2.897359856402041|
+------------------+



In [ ]:
# Question 2

In [7]:
trips_df.filter(trips_df.passenger_count == 2).agg({'trip_distance': 'mean'}).show( )

+------------------+
|avg(trip_distance)|
+------------------+
|2.9716795553859203|
+------------------+



In [ ]:
# Question 3

In [8]:
trips_df.filter(trips_df.RatecodeID == 2).count()

298564

In [ ]:
# Question 4

In [10]:
fare = trips_df.filter(trips_df.RatecodeID == 5).agg({'fare_amount': 'mean'}).show()

+------------------+
|  avg(fare_amount)|
+------------------+
|57.875639002215934|
+------------------+



In [4]:
import pyspark.sql.functions as F
trips_new= trips_df.withColumn(
    "date_diff_min", 
    (F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long"))/60.
)
trips_new.groupby('passenger_count').agg({'date_diff_min': 'mean'}).show()

+---------------+------------------+
|passenger_count|avg(date_diff_min)|
+---------------+------------------+
|           null|28.425607671552477|
|              1|15.532790958818948|
|              6|19.028427166873687|
|              3|17.459121473716532|
|              5|19.480482677562552|
|              9| 8.610493827160495|
|              4|17.449907643622797|
|              8|13.179047619047621|
|              7| 11.49060606060606|
|              2|16.361562048705718|
|              0|12.717738262886133|
+---------------+------------------+



In [5]:
trips_new2= trips_df.withColumn(
    "date_diff_hour", 
   (F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long"))/3600.
)
trips_new3= trips_new2.withColumn('speed', trips_new2.trip_distance/trips_new2.date_diff_hour)
trips_new3.groupby('RatecodeID').agg({'speed': 'mean'}).show()

+----------+------------------+
|RatecodeID|        avg(speed)|
+----------+------------------+
|      null|21.067977779855404|
|         1| 11.71248800540977|
|         6|317.85680681990516|
|         3| 53.24696907482183|
|         5| 77.86570552049292|
|         4| 45.09750559533565|
|         2| 34.46011888837545|
|        99| 7.502451564689387|
+----------+------------------+



# Part 2 - Pyspark Machine Learning

In [7]:
loans = spark.read.csv('gs://mhl-exam/data/loans2 (1).csv',header=True, inferSchema=True )

In [8]:
loans.count()

614

In [9]:
loans.head()

Row(Loan_ID='LP001002', Gender='Male', Married='No', Dependents='0', Education='Graduate', Self_Employed='No', ApplicantIncome=5849, CoapplicantIncome=0.0, LoanAmount=None, Loan_Amount_Term=360, Credit_History=1, Property_Area='Urban', Loan_Status='Y')

In [10]:
loans

DataFrame[Loan_ID: string, Gender: string, Married: string, Dependents: string, Education: string, Self_Employed: string, ApplicantIncome: int, CoapplicantIncome: double, LoanAmount: int, Loan_Amount_Term: int, Credit_History: int, Property_Area: string, Loan_Status: string]

In [11]:
df = loans.dropna()

In [12]:
df.show(3)

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y|
|LP001006|  Male|    Yes|         0|Not Graduate|           No|           2583|           2358.0|       120|             360|             1|        Urban|          Y

In [13]:
df = df.withColumn('total_income',(df.ApplicantIncome+df.CoapplicantIncome)).drop('ApplicantIncome','CoapplicantIncome')
df.show(3)

+--------+------+-------+----------+------------+-------------+----------+----------------+--------------+-------------+-----------+------------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|total_income|
+--------+------+-------+----------+------------+-------------+----------+----------------+--------------+-------------+-----------+------------+
|LP001003|  Male|    Yes|         1|    Graduate|           No|       128|             360|             1|        Rural|          N|      6091.0|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|        66|             360|             1|        Urban|          Y|      3000.0|
|LP001006|  Male|    Yes|         0|Not Graduate|           No|       120|             360|             1|        Urban|          Y|      4941.0|
+--------+------+-------+----------+------------+-------------+----------+----------------+--------------+-------------+----

In [15]:
df.agg({'total_income': 'mean'}).show()

+-----------------+
|avg(total_income)|
+-----------------+
|6945.324833309667|
+-----------------+



In [16]:
from pyspark.ml.feature import StringIndexer

In [23]:
indexer = StringIndexer(inputCol='Gender',outputCol='Gender_idx')
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)
indexer = StringIndexer(inputCol='Married',outputCol='Married_idx')
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)
indexer = StringIndexer(inputCol='Education',outputCol='Education_idx')
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)
indexer = StringIndexer(inputCol='Self_Employed',outputCol='Self_Employed_idx')
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)
indexer = StringIndexer(inputCol='Loan_Status',outputCol='Loan_Status_idx')
indexer_model = indexer.fit(df)
df = indexer_model.transform(df)

IllegalArgumentException: 'requirement failed: Output column Gender_idx already exists.'

In [24]:
df.agg({'Education_idx': 'mean'}).show()

+-------------------+
| avg(Education_idx)|
+-------------------+
|0.20208333333333334|
+-------------------+



In [21]:
from pyspark.ml.feature import OneHotEncoderEstimator

In [25]:
encoder = OneHotEncoderEstimator(inputCols=['Gender_idx','Married_idx','Education_idx','Self_Employed_idx'], outputCols=["Gender_vec","Married_vec","Education_vec","Self_Employed_vec"])

In [26]:
encoder_model = encoder.fit(df)
df = encoder_model.transform(df)
df = df.withColumnRenamed("Loan_Status_idx","label")

In [27]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['total_income','Gender_vec','Married_vec','Education_vec','Self_Employed_vec'],outputCol='features')
df = assembler.transform(df)

In [30]:
train, test = df.randomSplit([0.8,0.2], seed=2021)
[train.count(), test.count()]

[378, 102]

In [31]:
from pyspark.ml.classification import DecisionTreeClassifier

In [32]:
tree = DecisionTreeClassifier()
tree_model = tree.fit(train)
prediction = tree_model.transform(test)
prediction.select('label','prediction','probability','features').show(10, truncate=False)

+-----+----------+----------------------------------------+-------------------------+
|label|prediction|probability                             |features                 |
+-----+----------+----------------------------------------+-------------------------+
|0.0  |0.0       |[0.7407407407407407,0.25925925925925924]|[6000.0,1.0,0.0,1.0,1.0] |
|1.0  |0.0       |[0.7407407407407407,0.25925925925925924]|[4693.0,1.0,0.0,1.0,1.0] |
|1.0  |0.0       |[0.6458333333333334,0.3541666666666667] |[7660.0,1.0,1.0,0.0,1.0] |
|0.0  |1.0       |[0.1111111111111111,0.8888888888888888] |[11376.0,1.0,0.0,1.0,1.0]|
|1.0  |0.0       |[0.6363636363636364,0.36363636363636365]|[3600.0,1.0,0.0,1.0,1.0] |
|0.0  |0.0       |[0.7931034482758621,0.20689655172413793]|[6277.0,1.0,1.0,1.0,1.0] |
|0.0  |0.0       |[0.7931034482758621,0.20689655172413793]|[5821.0,1.0,1.0,1.0,1.0] |
|1.0  |0.0       |[0.7931034482758621,0.20689655172413793]|[11500.0,0.0,1.0,1.0,0.0]|
|0.0  |0.0       |[0.7931034482758621,0.20689655172413

In [33]:
prediction.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|    2|
|  0.0|       1.0|    4|
|  1.0|       0.0|   32|
|  0.0|       0.0|   64|
+-----+----------+-----+



In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [36]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print( "Accuracy = %g" % (accuracy))
print( "Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.647059
Test Error = 0.352941


In [37]:
results = prediction.select(['prediction', 'label'])

In [38]:
predictionAndLabels=results.rdd

In [39]:
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(predictionAndLabels)
recall = metrics.recall(label=1);
print("Recall for label=1: %g" % recall);

Recall for label=1: 0.0588235


In [40]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
weightedPrecision = evaluator.evaluate(prediction)
print( "weighted Precision = %g" % (weightedPrecision))

weighted Precision = 0.555556


In [41]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedRecall")
weightedRecall = evaluator.evaluate(prediction)
print( "weighted Recall = %g" % (weightedRecall))

weighted Recall = 0.647059
